In [2]:
import pandas as pd
import numpy as np
import regex as re
import string
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import Word2Vec
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

In [ ]:
import kagglehub

# Download latest version
data_path = kagglehub.dataset_download("manann/quotes-500k")

In [ ]:
df = pd.read_csv(data_path + "/quotes.csv")
df.columns = ["quote", "author", "category"]

In [ ]:
# most authors have a "," and then where the quote is from.
# So splitting with "," and keeping only the first part seems like the right choice.
df["author"] = df["author"]\
    .astype(str).str.split(",").\
    str[0].apply(lambda x: re.sub(f"[{string.punctuation}]", "", x)).\
    str.strip()

def preprocess_quote(text):
    # Convert the input to string to handle potential float values
    text = str(text)
    text = text.lower()
    text = re.sub(f"[{string.punctuation}]", "", text)
    tokens = text.split()
    tokens = [word for word in tokens if word not in ENGLISH_STOP_WORDS and len(word) >= 4]
    return " ".join(tokens)

df["processed_quote"] = df["quote"].apply(preprocess_quote)


# Group quotes by author
author_texts = df.groupby("author")["processed_quote"].apply(lambda x: " ".join(x)).to_dict()


# dropping all empty data
df = df.dropna(subset=["author", "quote"])

# grouping quotes by authors
author_texts = df.groupby("author")["quote"].apply(" ".join).to_dict()

def print_frequencies(author_texts, author):
